In [1]:
#导入相关的包
#如果没有使用pip安装
import pandas as pd
import numpy as np
import jieba
from sklearn.model_selection import train_test_split

In [2]:
#导入数据
#使用前请解压
news = pd.read_csv('sqlResult.csv', encoding='gb18030')
news.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [3]:
# 检查空值
news[news.content.isna()].head()

,id,author,source,content,feature,title,url
100,89517,NaN,中国证券报?中证网,NaN,"{""type"":""公司"",""site"":""中证网"",""commentNum"":""0"",""jo...",天和防务股东未来6个月内计划减持不超过480万股公司股份,http://www.cs.com.cn/ssgs/gsxw/201706/t2017062...
103,89514,NaN,中国证券报?中证网,NaN,"{""type"":""公司"",""site"":""中证网"",""commentNum"":""0"",""jo...",晶盛机电调整限制性股票回购价格,http://www.cs.com.cn/ssgs/gsxw/201706/t2017062...
997,88620,NaN,央广网,NaN,"{""type"":""时事要闻"",""site"":""参考消息"",""commentNum"":""0"",...",[主播不在家]第二季：主播陈亮体验垃圾清运,http://www.cankaoxiaoxi.com/china/20170623/214...
1273,88344,NaN,央广网,NaN,"{""type"":""IT业界"",""site"":""参考消息"",""commentNum"":""0"",...",LKK洛可可：想象力经济时代或已到来,http://www.cankaoxiaoxi.com/science/20170610/2...
1282,88335,NaN,央广网,NaN,"{""type"":""IT业界"",""site"":""参考消息"",""commentNum"":""0"",...",CES2017：京东发布两款叮咚智能音箱新品 开放Alpha平台,http://www.cankaoxiaoxi.com/science/20170610/2...


In [4]:
# 删除contene为空的行
news = news.dropna(subset=['content'])
news.shape
#可以看到有87054篇文章

(87054, 7)

In [5]:
# 加载中文停词表
# 停词对于文章表述没有太大的意义
stopwords = []
with open("chinese_stopwords.txt", 'r', encoding='utf-8') as ff:
    for line in ff:
        line = line.strip()
        stopwords.append(line)

# 自定义函数对content进行分词
# 因为中文分词相对英文来说困难一些 不过有中文分词表


def split_text(text):
    text = text.replace(' ', '').replace('\n', '').replace('\r', '')  # 去掉特殊符号
    textList = jieba.cut(text)  # 使用jieba模块对输入的段落进行切分
    result = ' '.join([i for i in textList if i not in stopwords])  # 去除停顿词
    return result


# 测试一下函数是否写的有问题
split_text(news.iloc[0].content)
# 分词写的是没有问题的

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\MUXIAO~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.862 seconds.
Prefix dict has been built successfully.


'此外 本周 除 小米 手机 款 机型 外 机型 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 集中 全部 精力 进行 系统优化 工作 有人 猜测 精力 主要 用到 MIUI9 研发 之中 MIUI8 去年 发布 距今已有 一年 有余 更新换代 当然 MIUI9 确切 信息 等待 官方消息'

In [6]:
# 构建训练的语料库
corpus = list(map(split_text, [str(i) for i in news.content]))
corpus

['此外 本周 除 小米 手机 款 机型 外 机型 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 确保 工程师 集中 全部 精力 进行 系统优化 工作 有人 猜测 精力 主要 用到 MIUI9 研发 之中 MIUI8 去年 发布 距今已有 一年 有余 更新换代 当然 MIUI9 确切 信息 等待 官方消息',
 '骁龙 835 唯一 Windows10 桌面 平台 认证 ARM 处理器 高通 强调 不会 只 考虑 性能 屏蔽掉 小 核心 相反 正 联手 微软 找到 一种 适合 桌面 平台 兼顾 性能 功耗 完美 方案 报道 微软 已经 拿到 一些 源码 Windows10 更好 理解 big little 架构 资料 显示 骁龙 835 一款 集成 CPU GPU 基带 蓝牙 Wi Fi SoC 传统 Wintel 方案 节省 至少 30% PCB 空间 按计划 今年 Q4 华硕 惠普 联想 首发 骁龙 835Win10 电脑 预计 均 二合一 形态 产品 当然 高通 骁龙 未来 也许 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 进入 Windows10 桌面 平台',
 '此前 一加 3T 搭载 3400mAh 电池 DashCharge 快充 规格 5V 4A 电池 缩水 可能 刘作 虎 所说 一加 手机 市面 轻薄 大屏 旗舰 设定 按照 目前 掌握 资料 一加 手机 拥有 5.5 寸 1080P 三星 AMOLED 显示屏 6G 8GBRAM 64GB 128GBROM 双 1600 万 摄像头 备货 量 惊喜 京东 泄露 信息 一加 售价 xx99 应该 2799 2899 2999',
 '这是 18 葡萄牙 中部 佩德罗 冈 地区 拍摄 森林 大火 烧毁 汽车 新华社 记者 张立 云摄',
 '原 标题 44 岁 女子 跑 深圳 约会 网友 拒 暴雨 裸身 奔走 … … @ 深圳 交警 微博称 昨日 清晨 交警 发现 女子 赤裸 上身 行走 南坪 快速 期间 轻生 年头 一辅警 发现 赶紧 披上 黄衣 一路 劝说 事发 到底 发生 些 南 记者 带 一起 还原 现场 南 记者 龙岗 大队 坂田 中队 见到 辅警 刘青 发现 女生 辅警 一位 外表 高大 帅气 说话 略带 些 腼腆 90 青年 刘青 介绍 16 早上 

In [7]:
#使用CountVectorizer提取文本的特征
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
# CountVectorizer是属于常见的特征数值计算类，是一个文本特征提取方法。对于每一个训练文本，它只考虑每种词汇在该训练文本中出现的频率
countvectorizer = CountVectorizer(encoding='gb18030', min_df=0.015)
tfidftransformer = TfidfTransformer()
countvect = countvectorizer.fit_transform(corpus)
tfidf = tfidftransformer.fit_transform(countvect)
tfidf.shape

(87054, 884)

In [8]:
# 标记是否为新华社的新闻 如果是新华社的新闻则标记为1 否则为0
label = list(map(lambda x: 1 if '新华社' in str(x) else 0, news.source))
label

[0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [9]:
# 进行数据集的切分
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
x_train, x_test, y_train, y_test = train_test_split(
    tfidf.toarray(), label, test_size=0.3, random_state=2020)
# 使用朴素贝叶斯
model = MultinomialNB()
model.fit(x_train, y_train)  # 进行训练
y_pre = model.predict(x_test)
print("准确率：", accuracy_score(y_test, y_pre))

准确率： 0.8850174216027874


In [10]:
# 使用模型对所有数据进行预测
prediction = model.predict(tfidf.toarray())
labels = np.array(label)
compare_news_data = pd.DataFrame({'prediction': prediction, 'labels': labels})
# prediction 为通过朴素贝叶斯预测的结果
# labels 为真实的结果
# 如果预测的结果为1 但真实的结果为0 我们就认为这篇文章有抄袭的可能
# 找到有抄袭可能的index
copyNewIndex = compare_news_data[(compare_news_data['prediction'] == 1) & (
    compare_news_data['labels'] == 0)].index
copyNewIndex

Int64Index([   4,   24,   25,   28,   30,   32,   35,   37,   38,   43,
            ...
            8535, 8538, 8539, 8543, 8544, 8546, 8548, 8549, 8551, 8552],
           dtype='int64', length=2835)

In [11]:
# 我们找出实际为新华社的新闻
trueNewsIndex = compare_news_data[(compare_news_data['labels'] == 1)].index
trueNewsIndex

Int64Index([    3,    39,    65,    95,   111,   303,   310,   312,   541,
              543,
            ...
            87044, 87045, 87046, 87047, 87048, 87049, 87050, 87051, 87052,
            87053],
           dtype='int64', length=78674)

In [12]:
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans

#进行标准化
normalizer=Normalizer()
scaled_array=normalizer.fit_transform(tfidf.toarray())

#使用Kmeans 进行聚类
kmeans=KMeans(n_clusters=25)
k_labels=kmeans.fit_predict(scaled_array)
k_labels.shape

(87054,)

In [13]:
from collections import defaultdict
id_class={index:class_ for index, class_ in enumerate(k_labels)}
class_id=defaultdict(set)
for index,class_ in id_class.items():
    if index in trueNewsIndex.tolist():
        class_id[class_].add(index)

In [14]:
class_id

defaultdict(set,
            {7: {3,
              65544,
              32785,
              32786,
              65553,
              65565,
              65566,
              65567,
              65572,
              65573,
              65574,
              39,
              65575,
              65587,
              65588,
              65589,
              65591,
              65593,
              32828,
              32830,
              32831,
              80760,
              32853,
              32854,
              32855,
              32856,
              32857,
              32858,
              65632,
              32866,
              78175,
              32871,
              65647,
              32880,
              32881,
              32882,
              32883,
              32884,
              32885,
              65648,
              65649,
              65650,
              65651,
              65652,
              82064,
              65663,
              65665,


In [15]:
#查找相似的文章
from sklearn.metrics.pairwise import cosine_similarity
def find_similar_text(cpindex,top=10):
    # 只在新华社发布的文章中查找
    dist_dict={i:cosine_similarity(tfidf[cpindex],tfidf[i]) for i in class_id[id_class[cpindex]]}
    #进行从大到小排序
    return sorted(dist_dict.items(),key=lambda x:x[1][0],reverse=True)[:top]

In [16]:
#选取index 为3352的 index 去查看相似度
similar_list=find_similar_text(3352)
similar_list

[(3134, array([[0.96849134]])),
 (63511, array([[0.94643202]])),
 (29441, array([[0.9428342]])),
 (3218, array([[0.87621844]])),
 (29615, array([[0.86936332]])),
 (29888, array([[0.86215823]])),
 (64046, array([[0.85278237]])),
 (64758, array([[0.73394727]])),
 (29973, array([[0.72524321]])),
 (15019, array([[0.6998852]]))]

In [17]:
news.iloc[3352].content

'\u3000\u3000中国5月份56座城市新建商品住宅价格环比上涨，4月份为58座上涨。5月份15个一线和热点二线城市房地产市场基本稳定，5月份房地产调控政策效果继续显现。\r\n\u3000\u3000统计局：15个一线和热点二线城市房价同比涨幅全部回落\r\n\u3000\u3000国家统计局城市司高级统计师刘建伟解读5月份房价数据\r\n\u3000\u30005月份一二线城市房价平均涨幅继续回落\r\n\u3000\u3000国家统计局今日发布了2017年5月份70个大中城市住宅销售价格统计数据。对此，国家统计局城市司高级统计师刘建伟进行了解读。\r\n\u3000\u3000一、15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落、9个城市环比下降或持平\r\n\u3000\u30005月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基本稳定。从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间。从环比看，9个城市新建商品住宅价格下降或持平；5个城市涨幅在0.5%以内。\r\n\u3000\u3000二、70个大中城市中一二线城市房价同比涨幅持续回落\r\n\u3000\u30005月份，70个城市中新建商品住宅和二手住宅价格同比涨幅比上月回落的城市分别有29和18个。其中，一二线城市同比涨幅回落尤其明显。据测算，一线城市新建商品住宅和二手住宅价格同比涨幅均连续8个月回落，5月份比4月份分别回落2.2和1.7个百分点；二线城市新建商品住宅和二手住宅价格同比涨幅分别连续6个月和4个月回落，5月份比4月份分别回落0.8和0.5个百分点。\r\n\u3000\u3000三、70个大中城市中房价环比下降及涨幅回落城市个数均有所增加\r\n\u3000\u30005月份，70个城市中新建商品住宅价格环比下降的城市有9个，比上月增加1个；涨幅回落的城市有26个，比上月增加3个。二手住宅价格环比下降的城市有7个，比上月增加2个；涨幅回落的城市有30个，比上月增加8个。\r\n'

In [18]:
news.iloc[similar_list[0][0]].content

'\u3000\u3000国家统计局19日发布数据，5月份，15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落，其中9个城市环比下降或持平。这9个价格环比下降或持平的城市为：北京、上海、南京、杭州、合肥、福州、郑州、深圳、成都。\r\n\u3000\u3000“5月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基本稳定。”国家统计局城市司高级统计师刘建伟说，从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间。从环比看，9个城市新建商品住宅价格下降或持平；5个城市涨幅在0.5%以内。\r\n\u3000\u3000国家统计局当天还发布了5月份70个大中城市住宅销售价格统计数据。刘建伟介绍，5月份，70个大中城市中新建商品住宅和二手住宅价格同比涨幅比上月回落的城市分别有29和18个。其中，一二线城市同比涨幅回落尤其明显。据测算，一线城市新建商品住宅和二手住宅价格同比涨幅均连续8个月回落，5月份比4月份分别回落2.2和1.7个百分点；二线城市新建商品住宅和二手住宅价格同比涨幅分别连续6个月和4个月回落，5月份比4月份分别回落0.8和0.5个百分点。\r\n\u3000\u3000此外，70个大中城市中房价环比下降及涨幅回落城市个数均有所增加。统计显示，5月份，70个大中城市中新建商品住宅价格环比下降的城市有9个，比上月增加1个；涨幅回落的城市有26个，比上月增加3个。二手住宅价格环比下降的城市有7个，比上月增加2个；涨幅回落的城市有30个，比上月增加8个。\r\n'